In [1]:
# Installing the required libraries
# !pip install python-docx
# !pip install pytesseract
# !pip install gtts
# !pip install opencv-python-headless
# !pip install opencv-python
# !pip install moviepy
# !pip install docx2python
# !pip install tinytag
# !pip install Pillow

In [2]:
# Importing the necessary modules
import os
import cv2
import docx
from docx import Document
from gtts import gTTS
from moviepy.editor import *
from docx2python import docx2python
from tinytag import TinyTag

In [3]:
# Reading the Word document

from PIL import Image

def read_word_docx(document_path):
    doc = Document(document_path)
    paragraphs = [para.text for para in doc.paragraphs if para.text != '']

    doc_result = docx2python(document_path)
    images = []

    for i, image_data in enumerate(doc_result.images.values()):
        temp_image_path = f'temp_image_{i}'
        image_path = f'image_{i}.png'
        
        with open(temp_image_path, 'wb') as f:
            f.write(image_data)
        
        img = Image.open(temp_image_path)
        width, height = img.size
        new_height = 720
        new_width = int((new_height / height) * width)
        resized_img = img.resize((new_width, new_height), Image.ANTIALIAS)
        resized_img.save(image_path)
        
        os.remove(temp_image_path)
        images.append(image_path)

    return paragraphs, images


document_path = '/Users/psumit/Documents/Career/Codespace/Projects/text-to-video/text-to-video-EN/test4page.docx'
paragraphs, images = read_word_docx(document_path)

### Step 4: Transcribing the text and create audio files.

In [4]:
# Step 4: Creating audio files
def mp3_duration(file_path):
    tag = TinyTag.get(file_path)
    return tag.duration

def create_audio_files(paragraphs, language='en'):
    audio_files = []
    for i, text in enumerate(paragraphs):
        audio_path = f'audio_{i}.mp3'
        tts = gTTS(text, lang=language)
        tts.save(audio_path)
        audio_files.append(audio_path)
    return audio_files

audio_files = create_audio_files(paragraphs)

### Step 5: Creating video

In [5]:
import textwrap

def wrap_text(text, max_width=50):
    lines = textwrap.wrap(text, width=max_width)
    return '\n'.join(lines)


def create_video(paragraphs, images, audio_files, output_path='/Users/psumit/Documents/Career/Codespace/Projects/text-to-video/text-to-video-EN/NewOutput.mp4'):
    clips = []

    intro_text = wrap_text(paragraphs[0])
    outro_text = wrap_text(paragraphs[-1])

    intro_clip = TextClip(intro_text, fontsize=24, color='white', size=(1280, 720), bg_color='black', print_cmd=False)
    intro_audio = AudioFileClip(audio_files[0])
    intro_clip = intro_clip.set_duration(intro_audio.duration).set_audio(intro_audio)
    clips.append(intro_clip)

    for i in range(1, len(paragraphs) - 1):
        image_path = images[i - 1]
        audio_path = audio_files[i]

        img = ImageClip(image_path)
        img = img.set_position('center').resize(height=720)

        composite_clip = CompositeVideoClip([img], size=(1280, 720))
        audio_clip = AudioFileClip(audio_path)

        composite_clip = composite_clip.set_duration(audio_clip.duration).set_audio(audio_clip)
        clips.append(composite_clip)

    outro_clip = TextClip(outro_text, fontsize=24, color='white', size=(1280, 720), bg_color='black', print_cmd=False)
    outro_audio = AudioFileClip(audio_files[-1])
    outro_clip = outro_clip.set_duration(outro_audio.duration).set_audio(outro_audio)
    clips.append(outro_clip)

    final_clip = concatenate_videoclips(clips)
    final_clip.write_videofile(output_path, codec='libx264', fps=24)

create_video(paragraphs, images, audio_files)

Moviepy - Building video /Users/psumit/Documents/Career/Codespace/Projects/text-to-video/NewOutput.mp4.
MoviePy - Writing audio in NewOutputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /Users/psumit/Documents/Career/Codespace/Projects/text-to-video/NewOutput.mp4



Moviepy - Done !
Moviepy - video ready /Users/psumit/Documents/Career/Codespace/Projects/text-to-video/NewOutput.mp4
